# Emotion Recognition Using Convoluted Neural Network

By: Aurelius Justin Lim

This notebook aims to create a CNN in order to recognition human emotions. The emotions to be experimented are anger, happiness, and sadness. To do this, the network is trained from a dataset taken from kaggle. A training, test and validation set is used to ensure the correctness and optimal performance of the model. Ultimately, the model created in the notebook is integrated into an application that allows users to upload a photo and allow the model to determine the illustrated emotion. 

**Human Face Emotions By SANIDHYAK** \
**Natural Human Face Images for Emotion Recognition By Sudarshan** \
Reference: \
https://www.kaggle.com/datasets/sanidhyak/human-face-emotions?resource=download \
https://www.kaggle.com/datasets/sudarshanvaidya/random-images-for-face-emotion-recognition?resource=download \

In [5]:

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [6]:
from preprocess import preprocess

command = preprocess()
command.run()



Data successfully split into train, validation, and test sets.
Train set: 2644 images - {'sad': 660, 'angry': 780, 'happy': 1204}
Validation set: 331 images - {'sad': 82, 'angry': 98, 'happy': 151}
Test set: 332 images - {'sad': 83, 'angry': 98, 'happy': 151}


In [7]:
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


Found 2643 images belonging to 3 classes.
Found 331 images belonging to 3 classes.
Found 332 images belonging to 3 classes.


In [8]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


Epoch 1/50
82/82 [==============================] - 113s 1s/step - loss: 1.1760 - accuracy: 0.4416 - val_loss: 1.0645 - val_accuracy: 0.4563
Epoch 2/50
82/82 [==============================] - 84s 1s/step - loss: 1.0614 - accuracy: 0.4653 - val_loss: 1.0540 - val_accuracy: 0.4688
Epoch 3/50
82/82 [==============================] - 82s 1s/step - loss: 1.0603 - accuracy: 0.4684 - val_loss: 1.0423 - val_accuracy: 0.4875
Epoch 4/50
82/82 [==============================] - 77s 943ms/step - loss: 1.0537 - accuracy: 0.4730 - val_loss: 1.0504 - val_accuracy: 0.4656
Epoch 5/50
82/82 [==============================] - 77s 939ms/step - loss: 1.0510 - accuracy: 0.4749 - val_loss: 1.0499 - val_accuracy: 0.4625
Epoch 6/50
82/82 [==============================] - 85s 1s/step - loss: 1.0486 - accuracy: 0.4718 - val_loss: 1.0484 - val_accuracy: 0.4594
Epoch 7/50
82/82 [==============================] - 99s 1s/step - loss: 1.0579 - accuracy: 0.4734 - val_loss: 1.0597 - val_accuracy: 0.4688
Epoch 8/50
82

In [10]:
print('Test Generator Samples:', test_generator.samples)
print('Test Generator Batch Size:', test_generator.batch_size)
if test_generator.samples > 0 and test_generator.batch_size > 0:
    steps = test_generator.samples // test_generator.batch_size
    steps = max(steps, 1)  # Ensure at least one step
else:
    raise ValueError("Test generator has invalid samples or batch size.")



test_loss, test_acc = model.evaluate(test_generator, steps=steps)
print('Final Test Loss:', test_loss)
print('Final Test Accuracy:', test_acc)

print('Validation Generator Samples:', validation_generator.samples)
print('Validation Generator Batch Size:', validation_generator.batch_size)
if validation_generator.samples > 0 and validation_generator.batch_size > 0:
    steps = validation_generator.samples // validation_generator.batch_size
    steps = max(steps, 1)  # Ensure at least one step
else:
    raise ValueError("Test generator has invalid samples or batch size.")



val_loss, val_acc = model.evaluate(validation_generator, steps=steps)
print('Final Validation Loss:', val_loss)
print('Final Validation Accuracy:', val_acc)


Test Generator Samples: 332
Test Generator Batch Size: 32
10/10 [==============================] - 7s 779ms/step - loss: 0.8216 - accuracy: 0.6406
Final Test Loss: 0.8215895891189575
Final Test Accuracy: 0.640625
Validation Generator Samples: 331
Validation Generator Batch Size: 32
10/10 [==============================] - 5s 472ms/step - loss: 0.8547 - accuracy: 0.5875
Final Validation Loss: 0.8546947240829468
Final Validation Accuracy: 0.5874999761581421


In [11]:
# # Define your CNN model function
# def create_model(hp):
#     model = tf.keras.models.Sequential()
    
#     # First Conv2D layer
#     model.add(tf.keras.layers.Conv2D(
#         filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
#         kernel_size=hp.Choice('conv_1_kernel_size', values=[3, 5]),
#         activation='relu',
#         input_shape=(150, 150, 3)
#     ))
#     model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

#     # Second Conv2D layer
#     model.add(tf.keras.layers.Conv2D(
#         filters=hp.Int('conv_2_filters', min_value=64, max_value=256, step=64),
#         kernel_size=hp.Choice('conv_2_kernel_size', values=[3, 5]),
#         activation='relu'
#     ))
#     model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

#     # Third Conv2D layer to match the original model structure
#     model.add(tf.keras.layers.Conv2D(
#         filters=hp.Int('conv_3_filters', min_value=128, max_value=512, step=128),
#         kernel_size=hp.Choice('conv_3_kernel_size', values=[3, 5]),
#         activation='relu'
#     ))
#     model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

#     # Fully connected layers
#     model.add(tf.keras.layers.Flatten())
#     model.add(tf.keras.layers.Dense(
#         units=hp.Int('dense_units', min_value=64, max_value=512, step=64),  # Adjusted range for more flexibility
#         activation='relu'
#     ))
#     model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))
#     model.add(tf.keras.layers.Dense(3, activation='softmax'))

#     # Compile the model
#     model.compile(
#         loss='categorical_crossentropy',
#         optimizer=tf.keras.optimizers.Adam(
#             learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
#         ),
#         metrics=['accuracy']
#     )

#     return model

# # Create a Hyperband Tuner
# tuner = kt.Hyperband(
#     create_model,
#     objective='val_accuracy',  # Ensure this matches the metric used
#     max_epochs=10,
#     factor=3,
#     directory='logs',
#     project_name='cnn_model',
#     overwrite=True
# )

# # Perform the hyperparameter search
# tuner.search(
#     train_generator,
#     validation_data=validation_generator,
#     epochs=5
# )

# # Print the best hyperparameters
# best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
# print("Best Hyperparameters:")
# for key, value in best_hyperparameters.values.items():
#     print(f"{key}: {value}")

In [12]:
# model = Sequential([
#     # First Conv2D layer with tuned filters and kernel size
#     Conv2D(128, (3, 3), activation='relu', input_shape=(150, 150, 3)),
#     MaxPooling2D(2, 2),
    
#     # Second Conv2D layer with tuned filters and kernel size
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
    
#     # Third Conv2D layer with tuned filters and kernel size
#     Conv2D(512, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
    
#     # Flatten layer
#     Flatten(),
    
#     # Dense layer with tuned units
#     Dense(128, activation='relu'),
    
#     # Dropout layer with tuned dropout rate
#     Dropout(0.1),
    
#     # Output layer
#     Dense(3, activation='softmax')
# ])

# # Compile the model with tuned learning rate
# model.compile(
#     optimizer=Adam(learning_rate=0.00042232),  # Best learning rate
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# # Fit the model using the tuned epochs
# history = model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // train_generator.batch_size,
#     epochs=50,  # Best number of epochs
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // validation_generator.batch_size
# )


In [13]:
model.save('ml-model.h5')


In [23]:
print(train_generator.class_indices)

{'angry': 0, 'happy': 1, 'sad': 2}
